In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import openpyxl
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
import os
from scipy.spatial.distance import pdist, squareform

In [2]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
processed_dir = TOP + 'data/processed/'
figures_dir = TOP + 'reports/figures/'

In [3]:
import glob

In [15]:
files = glob.glob(interim_dir+'/*.xlsx', 
                   recursive = True)
for file in files:
    print(file)

/home/grace/Documents/python/nts_pfas/data/interim/mediod_new_universe_061123.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/pfas flags_to_GP_TK_20230629.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/final_landscape_with_predicted_NAMs_091123.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/predicted_NAMs_091123.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/final_universe_categorisations_wtsca_active_constraints_061123.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/CCD_CDR2020_271022.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/Nuclear Receptor Confirmatory Flags v2.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/final_universe_wtest_track_071123.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/final_universe_wMaxMin_061123.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/final_updated_universe_101123.xlsx
/home/grace/Documents/python/nts_pfas/data/interim/final_universe_categorisations_wCDR_061123.xlsx
/home

In [5]:
df = pd.read_excel(interim_dir+'final_universe_wtest_track_071123.xlsx', index_col = [0])

In [8]:
test_track = df.columns.tolist()

In [6]:
df1 = pd.read_excel(interim_dir+'final_landscape_with_predicted_NAMs_091123.xlsx', index_col = [0])

In [9]:
nams = df1.columns.tolist()

In [12]:
[e for e in test_track if e not  in nams]

['HLC', 'WS_pred_mol/L', 'WS_pred_mg/L', 'test_track']

In [14]:
df1 = pd.merge(df1, df[['dtxsid', 'HLC', 'WS_pred_mol/L', 'WS_pred_mg/L', 'test_track']], on = 'dtxsid', how = 'left')

In [16]:
df2 = pd.read_excel(interim_dir+'final_updated_universe_101123.xlsx', index_col = [0])

In [17]:
all_df = df2.columns.tolist()

In [19]:
[e for e in df1.columns.tolist() if e not in all_df]

['HLC', 'WS_pred_mol/L', 'WS_pred_mg/L', 'test_track']

In [20]:
final_df = pd.merge(df2, df1[['dtxsid', 'HLC', 'WS_pred_mol/L', 'WS_pred_mg/L', 'test_track']], on = 'dtxsid', how = 'left')

In [24]:
final_df.drop(['cluster',
 'cluster_centroid',
 'centroid_x',
 'centroid2',
 'centroid_y',
 'analogues',
 'Dist',
 'level2_cluster_centroid',
 'cluster_d',
 'lvl2_centroid',
 'lvl2_centroid2',
 'lvl2_analogues',
 'lvl2_Dist',
 'prelim_final_centroid',
 'prelim_final_dist',
 'level3_cluster_centroid',
 'lvl_cluster_d',
 'lvl3_centroid',
 'lvl3_centroid2',
 'lvl3_analogues',
 'lvl3_Dist'], axis = 1, inplace = True)

In [26]:
final_df.drop(['casrn',
 'name', 'group_str', ], axis = 1, inplace = True)

In [29]:
final_df.rename(columns = {'tsca_centroid' :'tsca_centroid_tag' ,
 'tsca_centroid_id2' : 'tsca_centroid_id' ,'tsca_active_centroid' : 'tsca_active_centroid_tag',
 
 'tsca_active_centroid_id2' :'tsca_active_centroid_id' }, inplace = True)

In [30]:
mydict = {k: v for k,v in zip(final_df.columns, final_df.columns)}

In [31]:
mydict

{'dtxsid': 'dtxsid',
 'DSSTox_QC-Level': 'DSSTox_QC-Level',
 'Substance_Name': 'Substance_Name',
 'Substance_CASRN': 'Substance_CASRN',
 'Substance_Type': 'Substance_Type',
 'smiles': 'smiles',
 'Structure_InChI': 'Structure_InChI',
 'Structure_InChIKey': 'Structure_InChIKey',
 'Structure_Formula': 'Structure_Formula',
 'Structure_MolWt': 'Structure_MolWt',
 'Structure_SMILES_2D-QSAR': 'Structure_SMILES_2D-QSAR',
 'Canonical_QSARr': 'Canonical_QSARr',
 'TSCA_STATUS': 'TSCA_STATUS',
 'chain_length': 'chain_length',
 'category': 'category',
 'MolWeight': 'MolWeight',
 'LogP_pred': 'LogP_pred',
 'LogP_predRange': 'LogP_predRange',
 'AD_LogP': 'AD_LogP',
 'AD_index_LogP': 'AD_index_LogP',
 'Conf_index_LogP': 'Conf_index_LogP',
 'MP_pred': 'MP_pred',
 'MP_predRange': 'MP_predRange',
 'AD_MP': 'AD_MP',
 'AD_index_MP': 'AD_index_MP',
 'Conf_index_MP': 'Conf_index_MP',
 'BP_pred': 'BP_pred',
 'BP_predRange': 'BP_predRange',
 'AD_BP': 'AD_BP',
 'AD_index_BP': 'AD_index_BP',
 'Conf_index_BP': 'C

In [32]:
mydict = {'dtxsid': 'DSSTox Substance Identifier',
 'DSSTox_QC-Level': 'DSSTox_QC-Level',
 'Substance_Name': 'Chemical Name from DSSTox',
 'Substance_CASRN': 'CAS Registry Number',
 'Substance_Type': 'Substance_Type',
 'smiles': 'smiles',
 'Structure_InChI': 'Structure_InChI',
 'Structure_InChIKey': 'Hashed InChI',
 'Structure_Formula': 'Structure_Formula',
 'Structure_MolWt': 'Structure_MolWt',
 'Structure_SMILES_2D-QSAR': 'Structure_SMILES_2D-QSAR',
 'Canonical_QSARr': 'Canonical_QSARr',
 'TSCA_STATUS': 'TSCA Non-confidential lists per the Dashboard',
 'chain_length': 'Maximum number of contiguous CF2 groups in a chain',
 'category': 'primary category designation based on PFAS-Map assignment and simplified',
 'MolWeight': 'MolWeight',
 'LogP_pred': 'LogP_pred from OPERA v2.9',
 'LogP_predRange': 'LogP_predRange',
 'AD_LogP': 'Global Applicability Domain for the LogP model',
 'AD_index_LogP': 'Local Applicability Domain for the LogP model',
 'Conf_index_LogP': 'Conf_index_LogP',
 'MP_pred': 'MP_pred',
 'MP_predRange': 'MP_predRange',
 'AD_MP': 'AD_MP',
 'AD_index_MP': 'AD_index_MP',
 'Conf_index_MP': 'Conf_index_MP',
 'BP_pred': 'BP_pred',
 'BP_predRange': 'BP_predRange',
 'AD_BP': 'AD_BP',
 'AD_index_BP': 'AD_index_BP',
 'Conf_index_BP': 'Conf_index_BP',
 'LogVP_pred': 'LogVP_pred',
 'VP_predRange': 'VP_predRange',
 'AD_VP': 'AD_VP',
 'AD_index_VP': 'AD_index_VP',
 'Conf_index_VP': 'Conf_index_VP',
 'LogWS_pred': 'LogWS_pred',
 'WS_predRange': 'WS_predRange',
 'AD_WS': 'AD_WS',
 'AD_index_WS': 'AD_index_WS',
 'Conf_index_WS': 'Conf_index_WS',
 'LogHL_pred': 'LogHL_pred',
 'HL_predRange': 'HL_predRange',
 'AD_HL': 'AD_HL',
 'AD_index_HL': 'AD_index_HL',
 'Conf_index_HL': 'Conf_index_HL',
 'ID': 'Identifier to denote the name of the TSCA metabolites and where it originated from',
 'tsca_match': 'tsca_match',
 'subcategory': 'subcategory denoted by the greater or equal to 7 threshold, or less than 7',
 'final_centroid': 'DTXSID for the centroid substance per terminal category based on the entire universe',
 'final_dist': 'Pairwise Jaccard distance between a member of the terminal category and its corresponding centroid',
 'group': 'Terminal category description',
 'MaxMin_picks': 'MaxMin selections based on the entire universe of substances',
 'tsca_centroid_tag': 'tag to denote whether a substance was a centroid based on the constrained TSCA only universe',
 'tsca_centroid_id': 'DTXSID for the TSCA centroid substance',
 'tsca_analogues': 'tsca_analogues',
 'tsca_dist': 'Pairwise Jaccard distance between a member of the TSCA only terminal category and its TSCA centroid',
 'MaxMin_tsca_picks': 'MaxMin_tsca_picks',
 'tsca_active_centroid_tag': 'tsca_active_centroid_tag',
 'tsca_active_centroid_id': 'tsca_active_centroid_id',
 'tsca_active_analogues': 'tsca_active_analogues',
 'tsca_active_dist': 'Pairwise Jaccard distance between a member of the TSCA active only terminal category and its TSCA active centroid',
 'MaxMin_tsca_active_picks': 'MaxMin_tsca_active_picks',
 'CDR2019_NATIONAL_AGG_PV': 'Simplified tonnage taken from CDR website',
 'CDR2020_STATUS': 'CDR2020_STATUS',
 'qc': 'Analytic QC Score',
 'qc_httk':'TK analytical QC score to denote significant abiotic loss',
 'NR_ESR1': 'Nuclear Receptor Assay',
 'NR_PPAR': 'Nuclear Receptor Assay',
 'NR_NRF2': 'Nuclear Receptor Assay',
 'NR_RXR': 'Nuclear Receptor Assay',
 'NR_PXR': 'Nuclear Receptor Assay',
 'ZF': 'ZebraFish',
 'DNT': 'Developmental Neurotoxicity',
 'hDIO1': 'Thyroid Assay',
 'hDIO2': 'Thyroid Assay',
 'hDIO3': 'Thyroid Assay',
 'xDIO3': 'Thyroid Assay',
 'hIYD': 'Thyroid Assay',
 'XIYD': 'Thyroid Assay',
 'hTBG': 'Thyroid Assay',
 'hTPO': 'Thyroid Assay',
 'Immune': 'Bioseek',
 'TK_PlasBind': 'TK_PlasBind',
 'TK_Metab': 'TK_Metab',
 'TK_Struc_Endo': 'TK_Struc_Endo',
 'NAM': 'NAM',
 'pred_DNT': 'pred_DNT',
 'pred_Immune': 'pred_Immune',
 'pred_NR_ESR1': 'pred_NR_ESR1',
 'pred_NR_NRF2': 'pred_NR_NRF2',
 'pred_NR_PPAR': 'pred_NR_PPAR',
 'pred_NR_PXR': 'pred_NR_PXR',
 'pred_NR_RXR': 'pred_NR_RXR',
 'pred_No flag': 'pred_No flag',
 'pred_TK_PlasBind': 'pred_TK_PlasBind',
 'pred_ZF': 'pred_ZF',
 'pred_hDIO1': 'pred_hDIO1',
 'chronic': 'chronic data availability from ToxVal v9.4',
 'developmental': 'developmental data availability from ToxVal v9.4',
 'neurotoxicity': 'neurotoxicity data availability from ToxVal v9.4',
 'repeat dose other': 'repeat dose other',
 'reproduction': 'reproduction',
 'reproduction developmental': 'reproduction developmental',
 'short-term': 'short-term',
 'subchronic': 'subchronic',
 'ANALYTICAL_MTHDS':'PFAS with Validated Analytical Methods 533 and 537',
 'DISCHARGE': 'Discharge Monitoring Data - see https://echo.epa.gov/trends/pfas-tools#data',
 'DRINKING_WATER': 'Discharge Monitoring Data - see https://echo.epa.gov/trends/pfas-tools#data',
 'DRINKING_WATER-UCMR': 'DRINKING_WATER-UCMR',
 'PROD_DATA': 'Production Data see https://echo.epa.gov/trends/pfas-tools#data',
 'TRI-OFFSITE': 'Toxics Release Data see https://echo.epa.gov/trends/pfas-tools#data',
 'TRI-ONSITE': 'Toxics Release Data see https://echo.epa.gov/trends/pfas-tools#data',
 'TRI_WASTE': 'Toxics Release Data see https://echo.epa.gov/trends/pfas-tools#data',
 'pubmed_avail': 'pubmed search based on Names and CAS for the entire universe, 1 represents presence of an article',
 'HLC': 'HLC',
 'WS_pred_mol/L': 'WS_pred_mol/L',
 'WS_pred_mg/L': 'WS_pred_mg/L',
 'test_track': 'test_track'}

In [33]:
pd.DataFrame(mydict.items(), columns = ['column_name', 'explanation'])

,column_name,explanation
0,dtxsid,DSSTox Substance Identifier
1,DSSTox_QC-Level,DSSTox_QC-Level
2,Substance_Name,Chemical Name from DSSTox
3,Substance_CASRN,CAS Registry Number
4,Substance_Type,Substance_Type
...,...,...
114,pubmed_avail,pubmed search based on Names and CAS for the e...
115,HLC,HLC
116,WS_pred_mol/L,WS_pred_mol/L
117,WS_pred_mg/L,WS_pred_mg/L


In [34]:
writer = pd.ExcelWriter(interim_dir+'final_revised_universe_wmappingdict_101123.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.

final_df.to_excel(writer, sheet_name = 'universe')
pd.DataFrame(mydict.items(), columns = ['column_name', 'explanation']).to_excel(writer, sheet_name = 'mapping_dict')

writer.close()